# Working With STAC

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/master?filepath=docs%2Fexamples%2FWorking_with_STAC_simple.ipynb)

### STAC: SpatioTemporal Asset Catalog

> The SpatioTemporal Asset Catalog (STAC) specification aims to standardize the way geospatial assets are exposed online and queried. A 'spatiotemporal asset' is any file that represents information about the earth captured in a certain space and time. The initial focus is primarily remotely-sensed imagery (from satellites, but also planes, drones, balloons, etc), but the core is designed to be extensible to SAR, full motion video, point clouds, hyperspectral, LiDAR and derived data like NDVI, Digital Elevation Models, mosaics, etc.

Ref: https://github.com/radiantearth/stac-spechttps://github.com/radiantearth/stac-spec

Using STAC makes data indexation and discovery really easy. In addition to the Collection/Item/Asset (data) specifications, data providers are also encouraged to follow a STAC API specification:  https://github.com/radiantearth/stac-api-spec

> The API is compliant with the OGC API - Features standard (formerly known as OGC Web Feature Service 3), in that it defines many of the endpoints that STAC uses. A STAC API should be compatible and usable with any OGC API - Features clients. The STAC API can be thought of as a specialized Features API to search STAC Catalogs, where the features returned are STAC Items, that have common properties, links to their assets and geometries that represent the footprints of the geospatial assets.


## Requirements

To be able to run this notebook you'll need the following requirements:
- folium
- httpx
- rasterio

`!pip install folium httpx rasterio`

In [ ]:
# Uncomment this line if you need to install the dependencies
# !pip install folium requests rasterio

In [5]:
import httpx

from rasterio.features import bounds as featureBounds

from folium import Map, TileLayer, GeoJson

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [31]:
titiler_endpoint = "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.
stac_item = "https://sentinel-cogs.s3.amazonaws.com/sentinel-s2-l2a-cogs/34/S/GA/2020/3/S2A_34SGA_20200301_0_L2A/S2A_34SGA_20200301_0_L2A.json"

In [32]:
item = httpx.get(stac_item).json()
print(item)

{'type': 'Feature', 'stac_version': '1.0.0-beta.2', 'stac_extensions': ['eo', 'view', 'proj'], 'id': 'S2A_34SGA_20200301_0_L2A', 'bbox': [23.10608697239106, 31.512648251713085, 24.24775213279613, 32.51932567672379], 'geometry': {'type': 'Polygon', 'coordinates': [[[23.954137787703402, 31.512648251713085], [23.10608697239106, 31.529431170424147], [23.128869274501948, 32.51932567672379], [24.24775213279613, 32.495400678020594], [23.954137787703402, 31.512648251713085]]]}, 'properties': {'datetime': '2020-03-01T09:21:28Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'gsd': 10, 'view:off_nadir': 0, 'proj:epsg': 32634, 'sentinel:utm_zone': 34, 'sentinel:latitude_band': 'S', 'sentinel:grid_square': 'GA', 'sentinel:sequence': '0', 'sentinel:product_id': 'S2A_MSIL2A_20200301T090901_N0214_R050_T34SGA_20200301T114118', 'sentinel:data_coverage': 84.61, 'eo:cloud_cover': 1.81, 'sentinel:valid_cloud_cover': True}, 'collection': 'sentinel-s2-l2a-cogs', 'assets':

In [33]:
print(list(item["assets"]))
for it, asset in item["assets"].items():
    print(asset["type"])

['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL']
image/png
image/tiff; application=geotiff; profile=cloud-optimized
application/json
application/xml
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimized
image/tiff; application=geotiff; profile=cloud-optimize

In [34]:
bounds = featureBounds(item)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=8
)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

In [39]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/info",
    params = (
        ("url", stac_item),
        # Get info for multiple assets
        ("assets","visual"), ("assets","B04"), ("assets","B03"), ("assets","B02"),
    )
).json()
print(r)

{'visual': {'bounds': [23.106076243528157, 31.505173744374172, 24.296464503939948, 32.519334871696195], 'minzoom': 8, 'maxzoom': 14, 'band_metadata': [['1', {}], ['2', {}], ['3', {}]], 'band_descriptions': [['1', ''], ['2', ''], ['3', '']], 'dtype': 'uint8', 'nodata_type': 'Nodata', 'colorinterp': ['red', 'green', 'blue'], 'nodata_value': 0.0, 'count': 3, 'driver': 'GTiff', 'width': 10980, 'height': 10980, 'overviews': [2, 4, 8, 16]}, 'B04': {'bounds': [23.106076243528157, 31.505173744374172, 24.296464503939948, 32.519334871696195], 'minzoom': 8, 'maxzoom': 14, 'band_metadata': [['1', {}]], 'band_descriptions': [['1', '']], 'dtype': 'uint16', 'nodata_type': 'Nodata', 'colorinterp': ['gray'], 'nodata_value': 0.0, 'count': 1, 'driver': 'GTiff', 'width': 10980, 'height': 10980, 'overviews': [2, 4, 8, 16]}, 'B03': {'bounds': [23.106076243528157, 31.505173744374172, 24.296464503939948, 32.519334871696195], 'minzoom': 8, 'maxzoom': 14, 'band_metadata': [['1', {}]], 'band_descriptions': [['1'

### Display one asset

In [40]:
r = httpx.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "visual",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

### Select Indexes for assets

In [43]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = {
        "url": stac_item,
        "assets": "visual",
        "asset_bidx": "visual|3,1,2",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

In [49]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = (
        ("url", stac_item),
        ("assets", "B04"),
        ("assets", "B03"),
        ("assets", "B02"),
        ("asset_bidx", "B04|1"),  # There is only one band per asset for Sentinel
        ("asset_bidx", "B03|1"),  # There is only one band per asset for Sentinel
        ("asset_bidx", "B02|1"),  # There is only one band per asset for Sentinel
        ("minzoom", 8),
        ("maxzoom", 14),
        ("rescale", "0,10000"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

Merge Assets

In [46]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = (
        ("url", stac_item),
        ("assets", "B04"),
        ("assets", "B03"),
        ("assets", "B02"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("rescale", "0,10000"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m

Apply Expression between assets

In [48]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/tilejson.json",
    params = (
        ("url", stac_item),
        ("expression", "(B08-B04)/(B08+B04)"),  # NDVI
        ("rescale", "-1,1"),
        ("minzoom", 8),
        ("maxzoom", 14),
        ("colormap_name", "viridis"),
    )
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m